In [1]:
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import requests
import json
import datetime
import numpy as np

In [2]:
def send_email(subject, body, recipient_email):
    # 163邮箱的邮件服务器设置
    smtp_server = "smtp.163.com"
    smtp_port = 25
    sender_email = 'Wdp_bot@163.com'  # 请替换为您的163邮箱地址
    sender_password = "LWDTWSEHSHCZWVXZ"  # 请替换为您的163邮箱密码或授权码

    # 创建邮件内容
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = recipient_email
    message["Subject"] = subject
    message.attach(MIMEText(body, "plain"))

    # 连接邮件服务器并发送邮件
    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()  # 开启TLS加密
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, recipient_email, message.as_string())
        print("邮件发送成功！")
    except Exception as e:
        print("邮件发送失败:", str(e))

In [3]:
def get_two_digits(value):
    return "{:.2f}".format(value)

In [4]:
def get_value_from_dict(data, path):
    keys = path.split('.')
    current_data = data
    for key in keys:
        if key in current_data:
            current_data = current_data[key]
        else:
            return None
    return current_data

In [5]:
def data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce):
    # 发送GET请求
    response1 = requests.get(url1)
    response2 = requests.get(url2)
    # 检查响应状态码，如果状态码为200，则请求成功
    if response1.status_code and response2.status_code == 200:
        # 获取响应数据
        dict1 = response1.json()
        dict2 = response2.json()
        #print(dict2)
        # 处理响应数据
        price1 = get_value_from_dict(dict1, path1)
        price2 = get_value_from_dict(dict2, path2)
        price1h = get_value_from_dict(dict2, path1h)
        
        if token_name2 == "BIT_OP":
            price1 = float(price1[:12])
            price2 = float(price2[:12])
            price2 = price2/10000000000
        elif token_name2 == "PEPE_OP":
            price1 = float(price1[:10])
            price2 = float(price2[:10])
        else:
            price1 = float(price1[:6])
            price2 = float(price2[:6])
        
        if isinstance(price1h, str):    
            price1h = float(price1h[:6])
        if price1h is None:
            price1h = 0
        
        price_gap = abs((price1 - price2)/price2*100)
        global gap_array, mail_index
        gap_array[nonce] = get_two_digits(price_gap)
        print(token_name1, price1, token_name2, price2, "1h_change", price1h, gap_array[nonce])
        if price_gap > gap:
            subject = f"警报：{token_name2} potential arbitrage！"
            body = f"{current_time}:警报：{token_name2}代币出现套利空间，目前价差为{gap_array[nonce]},{token_name1}价格为{price1},{token_name2}价格为{price2}！"
            print(f"警报：{token_name2}代币出现套利空间，目前价差为{gap_array[nonce]},{token_name1}价格为{price1},{token_name2}价格为{price2}！")
            if mail_index[nonce] == 0 or mail_index[nonce] == 2:
                send_email(subject, body, recipient_email)
                mail_index[nonce] = 1
        elif abs(price1h) > 10:
            subject = f"警报：{token_name2} 价格波动超过10%"
            body = f"{current_time}:警报：{token_name2}代币价格出现巨大波动，目前1h变化为{price1h}！"
            print(f"警报：{token_name2}代币价格出现巨大波动，目前1h变化为{price1h}！")
            if mail_index[nonce] == 0:
                send_email(subject, body, recipient_email)
                mail_index[nonce] = 2
        else:
            if mail_index[nonce] == 1:
                subject = f"警报解除：{token_name2} 价差消除"
                body = f"{current_time}:警报解除：{token_name2}代币价差消除，目前价差为{gap_array[nonce]},{token_name1}价格为{price1},{token_name2}价格为{price2}！"
                print(f"{current_time}:警报解除：{token_name2}代币价差消除，目前价差为{gap_array[nonce]},{token_name1}价格为{price1},{token_name2}价格为{price2}！")
                send_email(subject, body, recipient_email)
                mail_index[nonce] = 0
            elif mail_index[nonce] == 2:
                subject = f"警报：{token_name2} 价格波动消除"
                body = f"{current_time}:警报：{token_name2}代币价格波动消除，目前1h变化为{price1h}！"
                print(f"警报：{token_name2}代币价格波动消除，目前1h变化为{price1h}！")
                send_email(subject, body, recipient_email)
                mail_index[nonce] = 0
            #else:
                #print(f"{token_name1}价差正常")
    else:
        print("请求失败，状态码：", response1.status_code, response2.status_code)

In [6]:
def MET_function():
    token_name1 = "MET_ETH"
    token_name2 = "MET_OP"
    url1 = "https://api.dexscreener.com/latest/dex/pairs/ethereum/0xceb5c29bde4604296135dd7b027a433fd3633516"
    url2 = "https://api.geckoterminal.com/api/v2/networks/optimism/pools/0xa8e4fa96327c5a93e159bd781f3ccfd860519d82"
    path1 = 'pair.priceUsd'
    path2 = 'data.attributes.base_token_price_usd'
    path1h = 'data.attributes.price_change_percentage.h1'
    global recipient_email
    gap = 15
    current_time = datetime.datetime.now()
    nonce = 1
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [7]:
def T_function():
    token_name1 = "T_ETH"
    token_name2 = "T_OP"
    url1 = "https://api.geckoterminal.com/api/v2/networks/eth/pools/0x752ebeb79963cf0732e9c0fec72a49fd1defaeac"
    url2 = "https://api.dexscreener.com/latest/dex/pairs/optimism/0xc60d585edf334bed161975c2bc91275123a70859"
    path1 = 'data.attributes.base_token_price_usd'
    path2 = 'pair.priceUsd'
    path1h = 'pair.priceChange.h1'
    global recipient_email
    gap = 20
    current_time = datetime.datetime.now()
    nonce = 2
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [8]:
def BIT_function():
    token_name1 = "BIT_ETH"
    token_name2 = "BIT_OP"
    url1 = "https://api.dexscreener.com/latest/dex/pairs/ethereum/0x2cc846fff0b08fb3bffad71f53a60b4b6e6d6482"
    url2 = "https://api.geckoterminal.com/api/v2/networks/optimism/pools/0x602ef41cf4f6a112f8e3dcbb7660287310a9fb51"
    path1 = 'pair.priceUsd'
    path2 = 'data.attributes.base_token_price_usd'
    path1h = 'data.attributes.price_change_percentage.h1'
    global recipient_email
    gap = 65
    current_time = datetime.datetime.now()
    nonce = 3
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [9]:
def PEPE_function():
    token_name1 = "PEPE_ETH"
    token_name2 = "PEPE_OP"
    url1 = "https://api.dexscreener.com/latest/dex/pairs/ethereum/0x11950d141ecb863f01007add7d1a342041227b58"
    url2 = "https://api.geckoterminal.com/api/v2/networks/optimism/pools/0x83b6f73ca05826b402343bc61d3c180e1597ae50"
    path1 = 'pair.priceUsd'
    path2 = 'data.attributes.base_token_price_usd'
    path1h = 'data.attributes.price_change_percentage.h1'
    global recipient_email
    gap = 30
    current_time = datetime.datetime.now()
    nonce = 4
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [10]:
def DHT_function():
    token_name1 = "DHT_ETH"
    token_name2 = "DHT_OP"
    url1 = "https://api.dexscreener.com/latest/dex/pairs/ethereum/0x303ffcD201831DB88422b76f633458e94E05C33e"
    url2 = "https://api.geckoterminal.com/api/v2/networks/optimism/pools/0x7e1357fa56c0ecfe69daf22e08489ae8de56d86c"
    path1 = 'pair.priceUsd'
    path2 = 'data.attributes.quote_token_price_usd'
    path1h = 'data.attributes.price_change_percentage.h1'
    global recipient_email
    gap = 20
    current_time = datetime.datetime.now()
    nonce = 5
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [11]:
def LYRA1_function():
    token_name1 = "lyra_ETH"
    token_name2 = "lyra_OP_celer"
    url1 = "https://api.dexscreener.com/latest/dex/pairs/ethereum/0x3012E36CC30eB7ed916B8D4b82750B39551874dC"
    url2 = "https://api.geckoterminal.com/api/v2/networks/optimism/pools/0xdb61f9b480f0a8b817811cfaa89a1c219c355224"
    path1 = 'pair.priceUsd'
    path2 = 'data.attributes.base_token_price_usd'
    path1h = 'data.attributes.price_change_percentage.h1'
    global recipient_email
    gap = 5
    current_time = datetime.datetime.now()
    nonce = 6
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [12]:
def LYRA2_function():
    token_name1 = "lyra_ETH"
    token_name2 = "lyra_ARB_celer"
    url1 = "https://api.dexscreener.com/latest/dex/pairs/ethereum/0x3012E36CC30eB7ed916B8D4b82750B39551874dC"
    url2 = "https://api.dexscreener.com/latest/dex/pairs/arbitrum/0x5Fa594Dd5e198DE5E7EF539F1b8fB154AeFD3891"
    path1 = 'pair.priceUsd'
    path2 = 'pair.priceUsd'
    path1h = 'pair.priceChange.h1'
    global recipient_email
    gap = 5
    current_time = datetime.datetime.now()
    nonce = 7
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [13]:
def THALES_function():
    token_name1 = "thales_arb"
    token_name2 = "thales_OP_celer"
    url1 = "https://api.dexscreener.com/latest/dex/pairs/arbitrum/0x8971dFb268B961a9270632f28B24F2f637c94244"
    url2 = "https://api.geckoterminal.com/api/v2/networks/optimism/pools/0xdff90e4a6c229565f25337b1db9fa12f6d8cb118"
    path1 = 'pair.priceUsd'
    path2 = 'data.attributes.base_token_price_usd'
    path1h = 'data.attributes.price_change_percentage.h1'
    global recipient_email
    gap = 5
    current_time = datetime.datetime.now()
    nonce = 8
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [14]:
def DF1_function():
    token_name1 = "DF_ETH"
    token_name2 = "DF_OP_celer"
    url1 = "https://api.dexscreener.com/latest/dex/pairs/ethereum/0x232818620877fd9232e9ADe0c91EF5518EB11788"
    url2 = "https://api.geckoterminal.com/api/v2/networks/optimism/pools/0x25c488e5f69ac334ac1e8f786d890a97ec1bf713"
    path1 = 'pair.priceUsd'
    path2 = 'data.attributes.base_token_price_usd'
    path1h = 'data.attributes.price_change_percentage.h1'
    global recipient_email
    gap = 15
    current_time = datetime.datetime.now()
    nonce = 9
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [15]:
def DF2_function():
    token_name1 = "DF_arb"
    token_name2 = "DF_OP_Celer"
    url1 = "https://api.dodoex.io/swap-data/pairs?apikey=5b17cd8cc739adefd0&limit=20&page=1&network=arbitrum&search=0x19E5910F61882Ff6605b576922507F1E1A0302FE&order_by=volume&order_direction=desc"
    url2 = "https://api.geckoterminal.com/api/v2/networks/optimism/pools/0x25c488e5f69ac334ac1e8f786d890a97ec1bf713"
    path1 = 'data.pairs.0xae6aab43c4f3e0cea4ab83752c278f8debaba689-0x641441c631e2f909700d2f41fd87f0aa6a6b4edb.basePrice'
    path2 = 'data.attributes.base_token_price_usd'
    path1h = 'data.attributes.price_change_percentage.h1'
    global recipient_email
    gap = 5
    current_time = datetime.datetime.now()
    nonce = 10
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [16]:
def KNC1_function():
    token_name1 = "KNC_ETH"
    token_name2 = "KNC_OP"
    url1 = "https://api.dexscreener.com/latest/dex/pairs/ethereum/0x1A262266364bDb34df4c1337AD5f8b9b57182251"
    url2 = "https://api.dexscreener.com/latest/dex/pairs/optimism/0xBf892C5175FD65003abC6980f0B6fEbE8dD06158"
    path1 = 'pair.priceUsd'
    path2 = 'pair.priceUsd'
    path1h = 'pair.priceChange.h1'
    global recipient_email
    gap = 8
    current_time = datetime.datetime.now()
    nonce = 11
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [17]:
def KNC2_function():
    token_name1 = "KNC_ETH"
    token_name2 = "KNC_arb"
    url1 = "https://api.dexscreener.com/latest/dex/pairs/ethereum/0x1A262266364bDb34df4c1337AD5f8b9b57182251"
    url2 = "https://api.dexscreener.com/latest/dex/pairs/arbitrum/0x96dF3539Db2E15088Ec4bf98d1b7f941e2c74159"
    path1 = 'pair.priceUsd'
    path2 = 'pair.priceUsd'
    path1h = 'pair.priceChange.h1'
    global recipient_email
    gap = 8
    current_time = datetime.datetime.now()
    nonce = 12
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [18]:
def UNIDX1_function():
    token_name1 = "UNIDX_ETH"
    token_name2 = "UNIDX_OP_synapse"
    url1 = "https://api.dexscreener.com/latest/dex/pairs/ethereum/0x3c6A0db9Bed03cc6c36c50C5090947F05f16de94"
    url2 = "https://api.geckoterminal.com/api/v2/networks/optimism/pools/0x9269f5199b01b7b7dd38321b85c3f34ee3f28f7f"
    path1 = 'pair.priceUsd'
    path2 = 'data.attributes.base_token_price_usd'
    path1h = 'data.attributes.price_change_percentage.h1'
    global recipient_email
    gap = 25
    current_time = datetime.datetime.now()
    nonce = 13
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [19]:
def UNIDX2_function():
    token_name1 = "UNIDX_ETH"
    token_name2 = "UNIDX_arb_synapse"
    url1 = "https://api.dexscreener.com/latest/dex/pairs/ethereum/0x3c6A0db9Bed03cc6c36c50C5090947F05f16de94"
    url2 = "https://api.dexscreener.com/latest/dex/pairs/arbitrum/0x7C9325298c7F30f53e02F4F48C5Ed374324b122E"
    path1 = 'pair.priceUsd'
    path2 = 'pair.priceUsd'
    path1h = 'pair.priceChange.h1'
    global recipient_email
    gap = 25
    current_time = datetime.datetime.now()
    nonce = 14
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [20]:
def SUSHI_function():
    token_name1 = "SUSHI_ETH"
    token_name2 = "SUSHI_OP"
    url1 = "https://api.dexscreener.com/latest/dex/pairs/ethereum/0x795065dcc9f64b5614c407a6efdc400da6221fb0"
    url2 = "https://api.geckoterminal.com/api/v2/networks/optimism/pools/0xe69a11c88517c1ab60e958de5d9096d93ef59802"
    path1 = 'pair.priceUsd'
    path2 = 'data.attributes.base_token_price_usd'
    path1h = 'data.attributes.price_change_percentage.h1'
    global recipient_email
    gap = 20
    current_time = datetime.datetime.now()
    nonce = 15
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [21]:
def PENDLE_function():
    token_name1 = "PENDLE_ETH"
    token_name2 = "PENDLE_OP"
    url1 = "https://api.dexscreener.com/latest/dex/pairs/ethereum/0x37922C69b08BABcCEaE735A31235c81f1d1e8E43"
    url2 = "https://api.geckoterminal.com/api/v2/networks/optimism/pools/0x7c9a7243f6a359781996822a141a28dc48aa258b"
    path1 = 'pair.priceUsd'
    path2 = 'data.attributes.base_token_price_usd'
    path1h = 'data.attributes.price_change_percentage.h1'
    global recipient_email
    gap = 8
    current_time = datetime.datetime.now()
    nonce = 16
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)
    

In [22]:
def ACX_function():
    token_name1 = "ACX_ETH"
    token_name2 = "ACX_OP"
    url1 = "https://api.dexscreener.com/latest/dex/pairs/ethereum/0x508acdC358be2ed126B1441F0Cff853dEc49d40F"
    url2 = "https://api.geckoterminal.com/api/v2/networks/optimism/pools/0x2cd32fb367431fd6f499328edf34934fdbbc6b26"
    path1 = 'pair.priceUsd'
    path2 = 'data.attributes.base_token_price_usd'
    path1h = 'data.attributes.price_change_percentage.h1'
    global recipient_email
    gap = 6
    current_time = datetime.datetime.now()
    nonce = 17
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [23]:
def OATH_function():
    token_name1 = "OATH_V1"
    token_name2 = "OATH_V2"
    url1 = "https://api.geckoterminal.com/api/v2/networks/optimism/pools/0xd20f6f1d8a675cdca155cb07b5dc9042c467153f"
    url2 = "https://api.geckoterminal.com/api/v2/networks/optimism/pools/0xc3439bc1a747e545887192d6b7f8be47f608473f"
    path1 = 'data.attributes.base_token_price_usd'
    path2 = 'data.attributes.base_token_price_usd'
    path1h = 'data.attributes.price_change_percentage.h1'
    global recipient_email
    gap = 5
    current_time = datetime.datetime.now()
    nonce = 18
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [24]:
def hop_function():
    token_name1 = "hop_base"
    token_name2 = "hop_op"
    url1 = "https://api.geckoterminal.com/api/v2/networks/base/pools/0xcc419f80ca98c36ca4013b1bd8c3294f65909ebb"
    url2 = "https://api.geckoterminal.com/api/v2/networks/optimism/pools/0x0e88a5bff34d39da66bdd32b82dc89a32d7bb6c9"
    path1 = 'data.attributes.base_token_price_usd'
    path2 = 'data.attributes.base_token_price_usd'
    path1h = 'data.attributes.price_change_percentage.h1'
    global recipient_email
    gap = 5
    current_time = datetime.datetime.now()
    nonce = 19
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [25]:
def main():
    try:
        MET_function()
        T_function()
        # BIT_function()
        # PEPE_function()
        DHT_function()
        LYRA1_function()
        LYRA2_function()
        THALES_function()
        # DF1_function()
        # DF2_function()
        KNC1_function()
        KNC2_function()
        # UNIDX1_function()
        # UNIDX2_function()
        SUSHI_function()
        PENDLE_function()
        ACX_function()
        OATH_function()
        hop_function()
        
        
        global index, err_index, token_array
        max_gap = max(gap_array)
        max_nonce = np.where(gap_array == max_gap)
        max_gap_token = token_array[max_nonce]
        current_time = datetime.datetime.now()
        index = index + 1
        err_index = 0
        print(f"rank:{index},now:{current_time},max gap token:{max_gap_token},gap_value:{max_gap}")
        print("\n")
        # 让程序休眠,单位为秒
        time.sleep(0.5 * 60)
        main()
    except Exception as e:
        index = 0
        err_index = err_index + 1
        print(f"err_rank:{err_index},Error: {e}")
        time.sleep(60)
        if err_index < 25:
            main()

In [26]:
recipient_email = "727226168@qq.com"
token_array = np.array(["None","MET","T","BIT","PEPE",'DHT','LYRA1','LYRA2','Thales','DF1','DF2','KNC1','KNC2','UNIDX1','UNIDX2','SUSHI','PENDLE','ACX','OATH','HOP'])
gap_array = np.zeros(30)
index = 0
err_index = 0
mail_index = np.zeros(30)
main()

MET_ETH 1.13 MET_OP 1.0381 1h_change 0.0 8.85
T_ETH 0.019 T_OP 0.0181 1h_change 0 4.97
DHT_ETH 0.0938 DHT_OP 0.0915 1h_change 0.0 2.51
lyra_ETH 0.0641 lyra_OP_celer 0.0652 1h_change 0.0 1.69
lyra_ETH 0.0641 lyra_ARB_celer 0.0643 1h_change 0 0.31
thales_arb 0.3761 thales_OP_celer 0.3794 1h_change 0.0 0.87
KNC_ETH 0.5163 KNC_OP 0.5328 1h_change -0.32 3.1
KNC_ETH 0.5163 KNC_arb 0.5386 1h_change 0 4.14
SUSHI_ETH 0.611 SUSHI_OP 0.5558 1h_change 0.0 9.93
PENDLE_ETH 0.7009 PENDLE_OP 0.6674 1h_change 0.0 5.02
ACX_ETH 0.053 ACX_OP 0.0516 1h_change 0.0 2.71
OATH_V1 0.0311 OATH_V2 0.0315 1h_change 0.0 1.27
OATH_base 0.0534 OATH_op 0.0538 1h_change 0.81 0.74
rank:1,now:2023-08-30 16:53:03.898863,max gap token:['SUSHI'],gap_value:9.93




KeyboardInterrupt: 